In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/tf/notebooks/src


## Intialization

### GPU

In [3]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Packages

In [4]:
!pip install pandas iterative-stratification nlpaug==0.0.20 tqdm click tensorflow_probability==0.11.1 tf2_resnets tensorflow_addons==0.11.1

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


### Imports

In [5]:
import os
import random
import logging
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from dataloader import BalancedMelSampler, MelSampler, csv_to_dict
from losses import NpairsLoss
from metrics import TFLWLRAP
from split_data import get_split
from train import get_model, get_callbacks, get_lr_metric
from models import NUM_FRAMES, Classifier, DeepMetricLearning

In [6]:
from params import *
from utils.logger import prepare_log_folder, create_logger

### Setup

In [7]:
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["SM_FRAMEWORK"] = "tf.keras"

physical_devices = tf.config.list_physical_devices("GPU")
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], True)
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

## Main

In [8]:
def get_lr(epoch, lr, min_lr=1e-5, max_lr=1e-3, epochs=30, warmup_prop=0.1):   
    if epoch <= epochs * warmup_prop:
        return min_lr + max_lr - (max_lr * (epochs * warmup_prop - epoch) / (epochs * warmup_prop))
    else:
        return min_lr + max_lr - (max_lr * (epoch - epochs * warmup_prop) / (epochs - epochs * warmup_prop))

In [9]:
def main(fold_idx, saved_path, pretrained_path, pretrained_with_contrastive):
    os.makedirs(os.path.join(saved_path, f"fold{fold_idx}"), exist_ok=True)
    pretrained_with_contrastive = bool(pretrained_with_contrastive)

    print(' -> Preparing Data \n')
    
    train_data = pd.read_csv("../data/new_train_tp.csv")
    train_index, val_index = get_split(fold=fold_idx)
    fold_train_dict = csv_to_dict(train_data.iloc[train_index])
    fold_valid_dict = csv_to_dict(train_data.iloc[val_index])

    balanced_train_data_loader = BalancedMelSampler(
        fold_train_dict,
        batch_size=32,
        max_length=NUM_FRAMES,
        is_train=True,
        n_classes=24,
        use_cutmix=True,
        cache=True,
        n_classes_in_batch=8,
        shuffle_aug=False,
    )

    valid_data_loader = MelSampler(
        fold_valid_dict,
        batch_size=balanced_train_data_loader.batch_size,
        n_classes=balanced_train_data_loader.n_classes,
        cache=True,
        max_length=NUM_FRAMES,
        is_train=False,
        use_cutmix=False,
        shuffle_aug=balanced_train_data_loader.shuffle_aug,
    )
    
#     return 0

    print(' -> Preparing Model \n')
    model = get_model(
        saved_path=saved_path,
        pretrained_with_contrastive=pretrained_with_contrastive,
        pretrained_path=pretrained_path,
    )
    model._build()
    
    scheduler = tfa.optimizers.Triangular2CyclicalLearningRate(
        initial_learning_rate=1e-4,
        maximal_learning_rate=1e-3,
        step_size=50,
    )
    
    optimizer = tf.keras.mixed_precision.experimental.LossScaleOptimizer(
        tfa.optimizers.Lookahead(
            tf.keras.optimizers.Adam(learning_rate=scheduler),
            10,
            0.5,
        ),
        "dynamic",
    )
    
    model.compile(
        optimizer=optimizer,
        metrics=[TFLWLRAP(num_classes=24, name="lwlrap")],
        metric_loss_fn=NpairsLoss(temperature=0.1, name="n_pairs"),
        classification_loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    )   

    print(' -> Training Model \n')

    callbacks = get_callbacks(pretrained_with_contrastive, fold_idx, saved_path=saved_path)
    steps_per_epoch = int((len(fold_train_dict)) / balanced_train_data_loader.batch_size)
    
    model.fit(
        balanced_train_data_loader,
        steps_per_epoch=steps_per_epoch,
        epochs=50,
        validation_data=valid_data_loader,
        callbacks=callbacks,
        verbose=2
    )

In [10]:
DEBUG = False
pretrained_folder = "../logs/2021-01-28/1/"   # simple
pretrained_folder = "../logs/2021-01-30/42/"  # mixstyle + iunterpolate + cbam
log_folder = "../logs/"

In [11]:
backbones = [
    "efficientnetb0",
    "efficientnetb1",
    "efficientnetb2",
    "efficientnetb3",
    "efficientnetb4",
    "resnet50",
]

In [12]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'Logging results to {log_folder}')
    create_logger(directory=log_folder, name="logs.txt")
    
for fold_idx in range(5):
    tf.keras.backend.clear_session()
    print(f"\n-------------   Fold {fold_idx + 1} / {5}  -------------\n")
    
    main(
        fold_idx, 
        log_folder, 
        pretrained_folder + f"pretrained_best_fold{fold_idx}.h5", 
        pretrained_with_contrastive=True
    )
    
    if DEBUG:
        break

Logging results to ../logs/2021-02-02/2/

-------------   Fold 1 / 5  -------------

 -> Preparing Data 

 -> Preparing Model 

 -> Training Model 

Epoch 1/50
Instructions for updating:
Use `tf.linalg.matmul` instead
30/30 - 10s - loss: 3.1524 - val_loss: 2.8889
Epoch 2/50
30/30 - 8s - loss: 3.1565 - val_loss: 2.8903
Epoch 3/50
30/30 - 8s - loss: 3.1801 - val_loss: 2.8909
Epoch 4/50
30/30 - 8s - loss: 3.1374 - val_loss: 2.8908
Epoch 5/50
30/30 - 8s - loss: 3.1743 - val_loss: 2.8908
Epoch 6/50
30/30 - 8s - loss: 3.0272 - val_loss: 2.8892
Epoch 7/50
30/30 - 8s - loss: 2.5629 - val_loss: 2.8895
Epoch 8/50
30/30 - 8s - loss: 2.3455 - val_loss: 2.8871
Epoch 9/50
30/30 - 8s - loss: 3.2284 - val_loss: 2.8807
Epoch 10/50
30/30 - 8s - loss: 3.1856 - val_loss: 2.8644
Epoch 11/50
30/30 - 8s - loss: 2.9422 - val_loss: 2.8051
Epoch 12/50
30/30 - 8s - loss: 3.0190 - val_loss: 2.6586
Epoch 13/50
30/30 - 8s - loss: 2.1792 - val_loss: 2.1697
Epoch 14/50
30/30 - 8s - loss: 1.8680 - val_loss: 1.8772
Epo

Epoch 25/50
30/30 - 8s - loss: 2.9757 - val_loss: 1.2612
Epoch 26/50
30/30 - 8s - loss: 2.8368 - val_loss: 1.2888
Epoch 27/50
30/30 - 8s - loss: 2.9844 - val_loss: 1.2928
Epoch 28/50
30/30 - 8s - loss: 1.7424 - val_loss: 1.3157
Epoch 29/50
30/30 - 8s - loss: 2.7566 - val_loss: 1.2660
Epoch 30/50
30/30 - 8s - loss: 1.7058 - val_loss: 1.3077
Epoch 31/50
30/30 - 8s - loss: 1.4695 - val_loss: 1.2617
Epoch 32/50
30/30 - 8s - loss: 1.6569 - val_loss: 1.2552
Epoch 33/50
30/30 - 8s - loss: 2.8912 - val_loss: 1.2703
Epoch 34/50
30/30 - 8s - loss: 1.6153 - val_loss: 1.2998
Epoch 35/50
30/30 - 8s - loss: 2.9695 - val_loss: 1.2327
Epoch 36/50
30/30 - 8s - loss: 2.9433 - val_loss: 1.3078
Epoch 37/50
30/30 - 8s - loss: 2.8730 - val_loss: 1.2811
Epoch 38/50
30/30 - 8s - loss: 1.5938 - val_loss: 1.3596
Epoch 39/50
30/30 - 8s - loss: 2.9218 - val_loss: 1.4400
Epoch 40/50
30/30 - 8s - loss: 2.9681 - val_loss: 1.3394
Epoch 41/50
30/30 - 8s - loss: 2.8957 - val_loss: 1.3204
Epoch 42/50
30/30 - 8s - loss: 